# Исследование надёжности заёмщиков

В ходе исследования необходимо разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок.

## Шаг 1. Откройте файл с данными и изучите общую информацию. 

Импортируем библиотеку

In [1]:
import pandas as pd

Прочитаем файл /datasets/data.csv и сохраним его в переменную df
Изучим общую информацию

In [2]:
df = pd.read_csv('/datasets/data.csv')

Выведем первые 10 строк таблицы

In [3]:
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Получим общую информацию о данных в таблице

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Рассмотрим полученную информацию

Всего в таблице 12 столбцов:

* children  — количество детей в семье 
* days_employed — общий трудовой стаж в днях
* dob_years — возраст клиента в годах
* education — уровень образования клиента
* education_id — идентификатор уровня образования
* family_status — семейное положение
* family_status_id — идентификатор семейного положения
* gender — пол клиента
* income_type — тип занятости
* debt — имел ли задолженность по возврату кредитов
* total_income — ежемесячный доход
* purpose — цель получения кредита

**Вывод**

Изучили таблицу со статисткой о платежеспособности заёмщиков, определили значения столбцов и нашли факторы, которые будут мешать анализу (нулевые строки, неверные типы данных столбцов). За счет чего, определили дальнейший план по работе с данными.

## Шаг 2. Предобработка данных

### Обработка пропусков

Проверим данные на наличие пропусков

In [5]:
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

В таблице присутствуют пропуски в столбцах с общим трудовым стажем (в днях) и с ежемесячным доходом. Прежде чем выбрать способ их устранения, решил проверить: зависят ли эти пропуски между собой?

In [6]:
 #делаем выборку по всем строкам, где значение стобца 'days_employed' пустое и считаем количество пустых значений в каждом слобце
df[df['days_employed'].isna()].isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Что мы получаем: Пропуски в столбцах взаимосвязаны, в каждой строке, где не заполнено значение одного из критерия не заполнено и второе. Осталось решить, как мы их будем заполнять.

Со значением ежемесячного дохода я буду поступать следующим образам: т.к. это один из ключевых факторов в принятии решения банка одобрять кредит или нет (чем выше з/п, тем более интересен банку человек, как заёмщик), будем ставить вместо пропусков отсутствие ежемесячного дохода, то есть 0 руб.

Со значением стажа немного другая история. Стаж можно спрогнозировать, хоть и не всегда правильно, т.к. нет определенного возраста, с которого человек начинает работать. Но это не единственный фактор, так как только это не могло бы нас оттолкнуть от "прогноза", ведь отклонение в 2-3 года при стаже в 15-30 лет можно считать не существенным. А вот фактор, который меня переубедил это делать, это тот факт, что человек, в 40 лет может уйти совершенно в другую область работы, и из-за этого стаж в этой отрасли у него будет накапливаться заново. Возможно, вы скажите, что это тоже не такой уж и большой процент. И, возможно, я с вами соглашусь. Но, взяв все во внимание, что пропуски в наших двух столбцах коррелирует, а в одоном столбце мы уже решили заменить пропуски з/п на её отсутствие, и, что задача у нас стоит: " разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок", а значит задачи искать связь между рабочим стажем и погашением кредита в срок у нас не стоит. Я считаю, что выбрал самое разумное решение.

В конце хочу подкрепить уверенностью моего решения, что, возможно, люди, которые не заполнили данных два столбца, не работали никогда и поэтому у них нет ни стажа, ни ежемесячного дохода

Заполняем пропуски в данных и смотрим на результат

In [7]:
df['days_employed'] = df['days_employed'].fillna(0)
df['total_income'] = df['total_income'].fillna(0)
df.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

**Вывод**

Выбрали способ заполнения пустых значений и использовали его. Теперь все ячейки в нашей таблицы заполнены, но это ещё не все...
Мы провели лишь первый шаг в предобработке данных

### Работа с аномалиями

Выведем общую информацию о наших значениях

In [8]:
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,2.152500e+04
mean,0.538908,56678.874622,43.293380,0.817236,0.972544,0.080883,1.505128e+05
std,1.381587,134870.763085,12.574584,0.548138,1.420324,0.272661,1.098972e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,0.000000e+00
25%,0.000000,-2518.168900,33.000000,1.000000,0.000000,0.000000,8.861283e+04
50%,0.000000,-982.531720,42.000000,1.000000,0.000000,0.000000,1.355147e+05
75%,1.000000,0.000000,53.000000,1.000000,1.000000,0.000000,1.955436e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


В глаза сразу бросаются отрицательные числа. В обоих колонках ('children', 'days_employed') отрицательные числа - не допустимы. Делаю предположение, что это частая ошибка в заполнения форм подобного рода, а заемщики просто хотели поставить знак 'тире' между вопросом и ответом. Так что предлагаю просто взять модуль от этих чисел.

In [9]:
df['children'] = abs(df['children'])
df['days_employed'] = abs(df['days_employed'])

In [39]:
df['children'].abs()

0        1
1        1
2        0
3        3
4        0
        ..
21449    1
21450    0
21451    1
21452    3
21453    2
Name: children, Length: 21454, dtype: int64

Посмотрим, как изменились наши данные

In [10]:
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,2.152500e+04
mean,0.543275,60156.419005,43.293380,0.817236,0.972544,0.080883,1.505128e+05
std,1.379876,133355.929525,12.574584,0.548138,1.420324,0.272661,1.098972e+05
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00
25%,0.000000,610.652074,33.000000,1.000000,0.000000,0.000000,8.861283e+04
50%,0.000000,1808.053434,42.000000,1.000000,0.000000,0.000000,1.355147e+05
75%,1.000000,4779.587738,53.000000,1.000000,1.000000,0.000000,1.955436e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


Вот и избавились мы от всех отрицательных чисел в наших данных. Но это была не единственная проблема. Конечно, огромные числа в столбце с ежемесячным доходом нас не удивляют, хоть и вызывают некоторые вопросы, но на столько большое количество дней трудового стажа явно говорят о не подлинности данных. Осталось лишь определить, что мы будим делать, с этими ошибками.

На сайте "РИА Новости" опубликована интересная статься, где говориться, что средний стаж выхода на пенсию составляет 34,5 лет (мы возьмем 35 лет). Так что предлагаю считать значения больше этого, как аномальные значения. А вот заменять мы их будем на среднее зачтение стажа в нашей таблице.

Использую метод apply(), чтобы заменить аномальный значения. 

In [11]:
days_employed_max = 35 * 365
days_employed_mean = (df[df['days_employed'] < days_employed_max])['days_employed'].mean()
df['days_employed'] = df['days_employed'].apply(lambda days_employed: days_employed_mean if days_employed > days_employed_max else days_employed)

In [12]:
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,2.152500e+04
mean,0.543275,2029.731359,43.293380,0.817236,0.972544,0.080883,1.505128e+05
std,1.379876,2000.171731,12.574584,0.548138,1.420324,0.272661,1.098972e+05
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00
25%,0.000000,610.652074,33.000000,1.000000,0.000000,0.000000,8.861283e+04
50%,0.000000,1808.053434,42.000000,1.000000,0.000000,0.000000,1.355147e+05
75%,1.000000,2491.158982,53.000000,1.000000,1.000000,0.000000,1.955436e+05
max,20.000000,12761.820562,75.000000,4.000000,4.000000,1.000000,2.265604e+06


**Вывод** 

Мы избавились от аномальных данных, теперь наша статистика будет точнее отражать результат.

### Замена типа данных

Посмотрим еще раз на тип данных каждого столбца

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Обратим внимание на тип данных столбцов 'days_employed' и 'total_income'. Они вещественного типа. Но значения дней и зарплаты удобнее хранить в целочисленных значениях.


Переопределим столбцы 'days_employed' и 'total_income' как целочисленные значения.

In [14]:
df['days_employed'] = df['days_employed'].astype('int')
df['total_income'] = df['total_income'].astype('int')

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


**Вывод**

Перевели значение столбцов 'days_employed' и 'total_income' из вещественного типа данных в целочисленный. Теперь наша талица намного красивее.

### Обработка дубликатов

Возможно, некоторые заявки у нас продублированы. Это может быть связано с многими факторами: сбой в системе; человек повторно подал заявку. Причина нам не интересно, ведь наша задача избавиться от них.

Сначала проверим, есть ли в нашей таблице эти дубликаты.

In [16]:
df.duplicated().sum()

54

Есть. Целых 54 шт. Возможно это и не сильно повлияет на нашу статистику, но в любом случае, мы не можем просто закрыть на это глаза. Так что уберем их. и Проверим количесетво повторяющихся строк.

Конечно, есть шанс, что это совершенно разные заявки, что два разных человека с одинаковыми характеристиками берут кредит, но я считаю, что это что-то из разряда фантастики. Так что считаю эти строки за дубликаты.

In [17]:
df = df.drop_duplicates().reset_index(drop = True)

In [18]:
df.duplicated().sum()

0

Прежде чем говорить о том, что мы избавились от всех дубликатов, взглянем ещё раз на значения в нашей таблице.

In [19]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,2029,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


Сразу бросаются значения в столбце 'education'. Давайте посмотрим, все значения, которые он в себе содержит.

In [20]:
df['education'].value_counts()

среднее                13705
высшее                  4710
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   273
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

Здесь одни и то же значения образования человека записаны в разном регистре, это могло содействовать потери дубликатов. Приведем все значения этого столбца к нижнему регистру и проведем проверку на дубликаты. В случае, если они появятся, уберем их.

In [21]:
df['education'] = df['education'].str.lower()

In [22]:
df['education'].value_counts()

среднее                15188
высшее                  5251
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

In [23]:
df.duplicated().sum()

17

In [24]:
df = df.drop_duplicates().reset_index(drop = True)

In [25]:
df.duplicated().sum()

0

**Вывод**

Избавились от строк, которые повторялись в нашей таблице. Нашли "неявные дубликаты", которые не видел метод duplicated() из-за написания значений в столбца 'education' в разном регистре. Привели все значение к единому регистру и избавились от найденных дубликатов

### Лемматизация и категоризация данных

Все время работы с данными о заемщиках, мне беспокоило, что цель запроса на кредит написана в свободной форме, ведь её можно разбить на определенные категории, а это даёт новые возможности.


Я решил найти ключевые слова в формулировка целей взятия кредиты, чтобы мы смогли их категорировать. 

Чтобы иметь какое-то представление о данных, выведем все значения, которые хранятся в нашей таблице.

In [26]:
df['purpose'].value_counts()

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
жилье                                     646
покупка жилья                             646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

В каждой из формулировки есть одно из этих слов: ['свадьба', 'недвижимость', 'жилье’, ‘автомобиль', 'образование'].
Из этих слов  вывеle 4 категории (Недвижимость и жилье берем как одну): ['на проведение свадьбы', 'операции с недвижимостью', 'на покупку автомобиля', 'для получения образования'].


Как мы это сделаем?
Создадим функцию, которая будет брать строку, смотреть значение в ячейке 'purpose', лемматизировать его и искать ключевые слова, которые мы уже выделили. По ним будет создавать новый столбец 'purpose_category'

In [27]:
# импортируем библиотеку pymystem3(преобразовывает слова к лемме):
from pymystem3 import Mystem
m = Mystem() 

In [28]:
#наша функция
def purpose_category(purpose):
    try:
        for word in purpose.split():
            word_lem = m.lemmatize(word)
            if 'свадьба' in word_lem:
                return 'на проведение свадьбы'
            if 'недвижимость' in word_lem:
                return 'операции с недвижимостью'
            if 'жилье' in word_lem:
                return 'операции с недвижимостью'
            if 'автомобиль' in word_lem:
                return 'на покупку автомобиля'
            if 'образование' in word_lem:
                return 'для получения образования'
    except:
        for word in purpose:
            word_lem = m.lemmatize(word)
            if 'свадьба' in word_lem:
                return 'на проведение свадьбы'
            if 'недвижимость' in word_lem:
                return 'операции с недвижимостью'
            if 'жилье' in word_lem:
                return 'операции с недвижимостью'
            if 'автомобиль' in word_lem:
                return 'на покупку автомобиля'
            if 'образование' in word_lem:
                return 'для получения образования'        

In [29]:
#применяем функцию к нашим данным и создаем столбец 'purpose_category'
df['purpose_category'] = df['purpose'].apply(purpose_category)

Функция отработала, теперь проверим, какие значения в себе хранит столбец 'purpose_category' и посмотрим на нашу таблицу

In [30]:
df['purpose_category'].value_counts()

операции с недвижимостью     10811
на покупку автомобиля         4306
для получения образования     4013
на проведение свадьбы         2324
Name: purpose_category, dtype: int64

In [31]:
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,операции с недвижимостью
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,на покупку автомобиля
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,операции с недвижимостью
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,для получения образования
4,0,2029,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,на проведение свадьбы
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,операции с недвижимостью
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,операции с недвижимостью
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,для получения образования
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,на проведение свадьбы
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,операции с недвижимостью


**Вывод**

С помощью лемматизации категорировали данные. На основе столбца 'purpose', вывели новый столбец 'purpose_category' с категориями цели обращения получения кредита. Создали функцию, которая, следуя алгоритма, относила данные к определенной категории, в функции использовали конструкцию try-except. С помощью метода apply() применили функцию к нашей структуре данных.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Для того, чтобы понять, есть ли зависимость между наличием детей и возвратом кредита в срок, сгруппировал данные по столбцу 'children' и среднему значению столбца 'debt'. Выведем полученный результат.

In [32]:
df.groupby('children')['debt'].mean().sort_values(ascending = False)

children
20    0.105263
4     0.097561
2     0.094542
1     0.091658
3     0.081818
0     0.075438
5     0.000000
Name: debt, dtype: float64

Процент выплаты кредита в срок у семей с 5ю детьми равен 0. Это очень странно, возможно, в нашей структуре мало случаев таких семей и все они не погасили кредит вовремя. Если это так, то эту строку можно не учитывать, давая ответ на нашу гипотезу.

In [33]:
len(df[df['children'] == 5])

9

**Вывод**

Что мы имеем: у супер-многодетных семей процент погашения кредита в срок больше, чем у других. А самый маленький шанс у семей с 5 детьми. Данными о семьях с 5ю детьми можно пренебречь, т.к. таких случаев всего 9. Таким образом, если не брать во внимание статистику по возврату денег семьями с 5ю детьми, можем сделать вывод, что семьи с детьми имеют больший шанс возврата кредита вовремя.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

Для того, чтобы понять, есть ли зависимость между семейным положением и возвратом кредита в срок, сгруппировал данные по столбцу 'family_status' и среднему значению столбца 'debt'. Отсортируем результат по убыванию и выведем его.

In [34]:
df.groupby('family_status')['debt'].mean().sort_values(ascending = False)

family_status
Не женат / не замужем    0.097509
гражданский брак         0.093471
женат / замужем          0.075452
в разводе                0.071130
вдовец / вдова           0.065693
Name: debt, dtype: float64

**Вывод**

На 1 месте находятся люди, не состоящие в браке. На 2 и 3 люди, которые находятся в отношениях. А на последние места попали люди, которые пережили расставание со своей парой. Можно вывести следующую зависимость. Человек, который ещё не заключил брак официально, имеет самый большой шанс вернуть кредит вовремя (между 1 и 2 места разница в 0.4%). А при расставании человек имеет самый маленький шанс возврата кредита в назначенный срок. Можно сделать вывод, что любовные отношения плохо влияют на возможность вернуть кредит (благосостояние), так что лучше в них не вступать (шутка, но закономерность все-таки есть).

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Чтобы вывести зависимость между уровнем дохода и возвратом кредита в срок. Разобьём нашу структуру данных на группы по доходу, посчитаем среднее значение столбца 'debt' и выведем на экран.

Поделим группы по доходу на 4 части. Для этого воспользуемся методом describe() и выберем диапазон для наших групп.

In [35]:
df['total_income'].describe()

count    2.145400e+04
mean     1.510105e+05
std      1.097366e+05
min      0.000000e+00
25%      8.908850e+04
50%      1.357810e+05
75%      1.958132e+05
max      2.265604e+06
Name: total_income, dtype: float64

Диапазон: [[0 - 90 000); [90 000 - 140 000); [140 000 - 200 000); [200 000- 2 700 000)].

Диапазон выбран, теперь группируем нашу структуру данных и выводим статистику.

In [36]:
df_25 = df[(df['total_income'] >= 0) & (df['total_income'] < 90000)]
df_50 = df[(df['total_income'] >= 90000) & (df['total_income'] < 140000)]
df_75 = df[(df['total_income'] >= 140000) & (df['total_income'] < 200000)]
df_100 = df[(df['total_income'] >= 200000) & (df['total_income'] < 2700000)]

In [37]:
print('total_income')
print('0 - 90 000:', '             {:.2%}'.format(df_25['debt'].mean()))
print('90 000 - 140 000:', '       {:.2%}'.format(df_50['debt'].mean()))
print('140 000 - 200 000:', '      {:.2%}'.format(df_75['debt'].mean()))
print('200 000 - 2 700 000:', '    {:.2%}'.format(df_100['debt'].mean()))

total_income
0 - 90 000:              7.93%
90 000 - 140 000:        8.41%
140 000 - 200 000:       9.02%
200 000 - 2 700 000:     7.07%


**Вывод**

Какой вывод можно сделать из данных. Прямой зависимости между уровнем дохода и возвратом кредита в срок не наблюдается. Видно лишь, что самые надёжные заёмщики - это люди, у которых доход - чуть выше среднего. А вот то, что удивляет, это то, что самый маленький процент погашения кредита в срок имеет группа с наивысшим доходом.

- Как разные цели кредита влияют на его возврат в срок?

In [38]:
df.groupby('purpose_category')['debt'].mean().sort_values(ascending = False)

purpose_category
на покупку автомобиля        0.093590
для получения образования    0.092200
на проведение свадьбы        0.080034
операции с недвижимостью     0.072334
Name: debt, dtype: float64

**Вывод**

В данной статистике видно, что выдача кретила 'на покупку автомобиля' и 'для получения образования' cамые надежные. А вот займ для 'операции с недвижимостью' люди чаще всего любят просрочивать. 

## Шаг 4. Общий вывод

Рабочие гипотезы:

* Есть ли зависимость между наличием детей и возвратом кредита в срок?
    
* Есть ли зависимость между семейным положением и возвратом кредита в срок?
    
* Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

* Как разные цели кредита влияют на его возврат в срок?

**Общие результаты**

Связать факт выплаченного или невыплаченного займа я попробовал со несколькими займами. И что же я выяснил:

* Люди без детей являются менее надежными заёмщиками.

* Чаще всего люди возвращают задолженность вовремя, не прочувствовав семейную жизнь. А реже всего те люди, чей брак стал недействительным по каким-либо причинам.

* Самое интересное я обнаружил здесь. Оказывается, самые богатые люди являются самыми ненадежными заёмщиками. А самыми надежными являются люди со средним достатком

* В гипотезе, где нужно связать цель получения кредита с фактом возврата в срок, я не смог дать разумное объяснение или закономерность, поэтому могу лишь констатировать факт. А заключается он в следующем: Люди которые берут кредит на машину или для получения образование чаще других погашают задолженность в срок. За ними в списке рейтинга надёжных заёмщиков идут люди, которые берут кредит на свадьбу, это можно попробовать объяснить тем, что после свадьбы, молодожены устраивают свой быт, а на это, обычно, уходит много ресурсов. Ну и самые ненадежные идут люди, берущие квартиру в кредит. Мне пока сложно объяснить почему так происходит, как возьму ипотеку и выплачу дополню проект.
